# Sythetic Multilabel Classification

## Data Generation
 y <== L dimension one-hot vector, each entry represent a label
 
 X <== y + N(0, $\sigma$)
 
 ## Train Process
 
* y $\in [0,1]^L$
 
* $\bar y = sign(M\cdot y) \in [0,1]^{\bar L}$, where M is a iid gaussian entry embedding matrix (store $\bar Y$ into local files for Matlab)
 
* $\tilde y = \textbf{BCHencode}(\bar y) \in [0,1]^{\tilde L}$ (need to use Matlab)
 
* Train multi-label random forest on $X, \tilde y$ 

**Notice**: for the BCH code, we choose the message length to be 67, codeword length to be 511, the error correction bit is 87. 

The error correction rate is 0.17

## Training Process

In [1]:
import numpy as np
from numpy.random import binomial
from numpy.random import normal
from numpy.random import randint
import numpy as np
np.random.seed(42)

In [2]:
# constants
SPARSE = 0.05 # sparsity of label vectors
SIGMA = 0.5 # standard diveation of noise
L = 1000 # feature and label dimension
N = 10000 # number of data points

L_bar =45 # embedding dimension, also the message length for BCH code
L_tilde = 255 # codeword length for BCH encoder

In [3]:
# generate synthetic data
y = binomial(1, SPARSE, size=(N, L)) # iid Bernoulli entries
X = y + normal(loc=0, scale=SIGMA, size=(N, L))
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33)

In [4]:
# Source encode + KNN searcher
M = normal(size=[L, L_bar])
y_train_bar = (np.sign(y_train.dot(M))+1)/2
import faiss
nn_index = faiss.index_factory(y_train_bar.shape[1], "Flat", faiss.METRIC_L2)   # build the index
nn_index.add(y_train_bar.astype('float32'))

Failed to load GPU Faiss: No module named swigfaiss_gpu
Faiss falling back to CPU-only.


In [5]:
# save y_bar to matlab file
from scipy.io import savemat, loadmat
savemat(file_name="../.temp/train/y_bar", mdict={'y_bars':[y_train_bar],
                                                 'L_tilde':L_tilde
                                                })

----

Using **Matlab** to encode $\bar y$ into $\tilde y$ ...

----

In [10]:
# load the y_tilde file generated by matlab
y_tildes = loadmat("../.temp/train/y_tilde")['y_tildes'].astype(int)
y_train_tilde = y_tildes[0]

In [ ]:
# train the random forest multi-label classifier
from pytictoc import TicToc
time = TicToc()
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_jobs=-1, n_estimators=100)

time.tic()
clf.fit(X_train, y_train_tilde)
time.toc("train random forest")